In [1]:
import os
import numpy as np
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision

import torch.nn.utils.spectral_norm as spectralnorm
import torchvision
import matplotlib.pyplot as plt
from PIL import Image
#from models import *
from utils import *
from helper import *
import seaborn as sns

device = torch.device('cuda' if torch.cuda.is_available else cpu)
from vimeo90k import Vimeo90kDataset, VideoFolder_diffusion
!nvidia-smi

Tue May  7 16:38:06 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     On   | 00000000:2F:00.0 Off |                    0 |
| N/A   34C    P8    23W / 250W |      0MiB / 23040MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import numpy as np
from torch.autograd import Variable
import torch.autograd as autograd
device = torch.device('cuda' if torch.cuda.is_available else cpu)
from models import Discriminator_v3
from ssf_model import ScaleSpaceFlow
import time

LAMBDA_GP = 50
LAMBDA_MSE = 1
EPOCHS = 50

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0),  1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake[:],
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

train_transforms = transforms.Compose(
        [transforms.ToTensor(), transforms.RandomCrop(256)])

train_dataset = VideoFolder_diffusion(
        "./data/vimeo-90k/vimeo_triplet/",
        rnd_interval=False,
        rnd_temp_order=False,
        split="train",
        transform=train_transforms)

train_dataloader = DataLoader(
        train_dataset,
        batch_size=64,
        shuffle=True,
        pin_memory=True)

discriminator_FMD = Discriminator_v3(ch=256, out_ch=3).to(device)
#discriminator_FMD.load_state_dict(torch.load('./saved_models/vimeo-90k/discriminator_FMD.pth'))
opt_FMD = torch.optim.Adam(discriminator_FMD.parameters(), lr=1e-6)

a = time.time()
for epoch in range(EPOCHS):
    FMD_list = []
    FMDD_epoch = 0
    discriminator_FMD.train()
    for i, data in enumerate(train_dataloader):
        if i%100 == 0:
            print(f'batch {i} of {len(train_dataloader)}')
        opt_FMD.zero_grad()        
        x1 = 2*(data[:,0,...]-0.5)
        x2 = 2*(data[:,1,...]-0.5)
        x1_hat = 2*(data[:,3,...]-0.5)
        x1=x1.to(device)
        x2=x2.to(device)
        x1_hat = x1_hat.to(device)
        
        #change code
        real_img = x2
        real_img = real_img[:32]
        fake_img = real_img[32:]
        fake_validity = discriminator_FMD(fake_img.detach())
        real_validity = discriminator_FMD(real_img)
        
        #gradient_penalty = compute_gradient_penalty(discriminator, real_vid.data, fake_vid.data)
        errFMD =  -torch.mean(real_validity) + torch.mean(fake_validity) #+ LAMBDA_GP * gradient_penalty
        errFMD.backward()
        opt_FMD.step()
        FMD_list.append(errFMD.item())        
            
    FMD_epoch = torch.Tensor(FMD_list).mean().item()
    b = time.time()
    run_time = (b-a)/60
    print(f'| EPOCH: {epoch} | FMD PLF: {FMD_epoch} TIME: {run_time} min|')
          
    if epoch % 1 == 0:
        print('saving models...')
        discriminator_FMD.eval()
        torch.save(discriminator_FMD.state_dict(), os.path.join("./saved_models/vimeo-90k/discriminator_FMD.pth"))

batch 0 of 802
batch 100 of 802
batch 200 of 802
batch 300 of 802
batch 400 of 802
batch 500 of 802
batch 600 of 802
batch 700 of 802
batch 800 of 802
| EPOCH: 0 | FMD PLF: nan TIME: 38.01817898352941 min|
saving models...
batch 0 of 802
batch 100 of 802


KeyboardInterrupt: 

In [4]:
print(torch.Tensor(FMD_list).mean().item())

nan


In [5]:
print(torch.Tensor(FMD_list))

tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan])
